In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
#import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB6

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

In [8]:
size=(120, 120)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [9]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            imgs = [randomErase(img) for img in imgs]
            pass
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [10]:
def getMultiBased():
    model =  Xception(weights="imagenet", include_top=False,input_shape=(*size,3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    #x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)
del df

In [12]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-xce-epoch{epoch:04}.h5")

In [ ]:
batch_size=256
train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size,size=size)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size,size=size)
#strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope():
model = getMultiBased()
model.summary()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint], workers=64, use_multiprocessing=True)
model.save("multiXception.h5")

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 59, 59, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 59, 59, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 59, 59, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

Epoch 1/80
706/707 [============================>.] - ETA: 0s - loss: 2.0873 - dense_1_loss: 3.1427 - dense_2_loss: 1.2870 - dense_3_loss: 0.7768 - dense_1_acc: 0.4673 - dense_2_acc: 0.6836 - dense_3_acc: 0.7938

707/707 [==============================] - 667s 944ms/step - loss: 2.0868 - dense_1_loss: 3.1420 - dense_2_loss: 1.2867 - dense_3_loss: 0.7767 - dense_1_acc: 0.4675 - dense_2_acc: 0.6837 - dense_3_acc: 0.7939 - val_loss: 0.2951 - val_dense_1_loss: 0.5345 - val_dense_2_loss: 0.2373 - val_dense_3_loss: 0.1682 - val_dense_1_acc: 0.8788 - val_dense_2_acc: 0.9519 - val_dense_3_acc: 0.9529
Epoch 2/80
707/707 [==============================] - 618s 874ms/step - loss: 1.6628 - dense_1_loss: 2.4564 - dense_2_loss: 1.0764 - dense_3_loss: 0.6620 - dense_1_acc: 0.5948 - dense_2_acc: 0.7237 - dense_3_acc: 0.8187 - val_loss: 0.2507 - val_dense_1_loss: 0.3816 - val_dense_2_loss: 0.1947 - val_dense_3_loss: 0.1316 - val_dense_1_acc: 0.8959 - val_dense_2_acc: 0.9507 - val_dense_3_acc: 0.9631
Ep

707/707 [==============================] - 618s 875ms/step - loss: 0.8962 - dense_1_loss: 1.1957 - dense_2_loss: 0.7115 - dense_3_loss: 0.4819 - dense_1_acc: 0.7240 - dense_2_acc: 0.8653 - dense_3_acc: 0.8887 - val_loss: 0.1008 - val_dense_1_loss: 0.1745 - val_dense_2_loss: 0.0569 - val_dense_3_loss: 0.0549 - val_dense_1_acc: 0.9521 - val_dense_2_acc: 0.9865 - val_dense_3_acc: 0.9863
Epoch 22/80
707/707 [==============================] - 619s 875ms/step - loss: 0.8845 - dense_1_loss: 1.1768 - dense_2_loss: 0.7066 - dense_3_loss: 0.4779 - dense_1_acc: 0.7316 - dense_2_acc: 0.8695 - dense_3_acc: 0.8906 - val_loss: 0.1158 - val_dense_1_loss: 0.1852 - val_dense_2_loss: 0.0679 - val_dense_3_loss: 0.0602 - val_dense_1_acc: 0.9528 - val_dense_2_acc: 0.9861 - val_dense_3_acc: 0.9857
Epoch 23/80
707/707 [==============================] - 619s 876ms/step - loss: 0.8814 - dense_1_loss: 1.1722 - dense_2_loss: 0.7041 - dense_3_loss: 0.4770 - dense_1_acc: 0.7325 - dense_2_acc: 0.8719 - dense_3_acc: 

Epoch 42/80
707/707 [==============================] - 620s 877ms/step - loss: 0.7798 - dense_1_loss: 1.0085 - dense_2_loss: 0.6580 - dense_3_loss: 0.4444 - dense_1_acc: 0.7901 - dense_2_acc: 0.9048 - dense_3_acc: 0.9227 - val_loss: 0.1180 - val_dense_1_loss: 0.1786 - val_dense_2_loss: 0.0549 - val_dense_3_loss: 0.0552 - val_dense_1_acc: 0.9533 - val_dense_2_acc: 0.9881 - val_dense_3_acc: 0.9875
Epoch 43/80
267/707 [==========>...................] - ETA: 6:10 - loss: 0.7794 - dense_1_loss: 1.0072 - dense_2_loss: 0.6549 - dense_3_loss: 0.4486 - dense_1_acc: 0.7913 - dense_2_acc: 0.9033 - dense_3_acc: 0.9216

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



707/707 [==============================] - 619s 876ms/step - loss: 0.7726 - dense_1_loss: 0.9976 - dense_2_loss: 0.6534 - dense_3_loss: 0.4418 - dense_1_acc: 0.7950 - dense_2_acc: 0.9073 - dense_3_acc: 0.9272 - val_loss: 0.1271 - val_dense_1_loss: 0.1948 - val_dense_2_loss: 0.0576 - val_dense_3_loss: 0.0497 - val_dense_1_acc: 0.9534 - val_dense_2_acc: 0.9864 - val_dense_3_acc: 0.9874
Epoch 46/80
707/707 [==============================] - 619s 876ms/step - loss: 0.7700 - dense_1_loss: 0.9940 - dense_2_loss: 0.6509 - dense_3_loss: 0.4413 - dense_1_acc: 0.7967 - dense_2_acc: 0.9100 - dense_3_acc: 0.9264 - val_loss: 0.0827 - val_dense_1_loss: 0.1831 - val_dense_2_loss: 0.0610 - val_dense_3_loss: 0.0507 - val_dense_1_acc: 0.9525 - val_dense_2_acc: 0.9862 - val_dense_3_acc: 0.9874
Epoch 47/80
707/707 [==============================] - 620s 877ms/step - loss: 0.7637 - dense_1_loss: 0.9828 - dense_2_loss: 0.6505 - dense_3_loss: 0.4387 - dense_1_acc: 0.7985 - dense_2_acc: 0.9089 - dense_3_acc: 

Epoch 66/80
707/707 [==============================] - 619s 876ms/step - loss: 0.7167 - dense_1_loss: 0.9080 - dense_2_loss: 0.6283 - dense_3_loss: 0.4223 - dense_1_acc: 0.8305 - dense_2_acc: 0.9230 - dense_3_acc: 0.9388 - val_loss: 0.1179 - val_dense_1_loss: 0.1827 - val_dense_2_loss: 0.0681 - val_dense_3_loss: 0.0551 - val_dense_1_acc: 0.9551 - val_dense_2_acc: 0.9873 - val_dense_3_acc: 0.9870
Epoch 67/80
707/707 [==============================] - 620s 876ms/step - loss: 0.7182 - dense_1_loss: 0.9094 - dense_2_loss: 0.6301 - dense_3_loss: 0.4238 - dense_1_acc: 0.8306 - dense_2_acc: 0.9241 - dense_3_acc: 0.9409 - val_loss: 0.0882 - val_dense_1_loss: 0.1787 - val_dense_2_loss: 0.0513 - val_dense_3_loss: 0.0471 - val_dense_1_acc: 0.9567 - val_dense_2_acc: 0.9878 - val_dense_3_acc: 0.9884
Epoch 68/80
707/707 [==============================] - 619s 875ms/step - loss: 0.7168 - dense_1_loss: 0.9081 - dense_2_loss: 0.6278 - dense_3_loss: 0.4232 - dense_1_acc: 0.8307 - dense_2_acc: 0.9254 - d